In [ ]:
import openai, json

client = openai.OpenAI()
messages = []

# 그냥 아무거나 리턴하는 의미 없는 함수
def get_weather(city):
    return "33 degrees celcius."

FUNCTION_MAP = {
    'get_weather': get_weather,
}

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the weather in a given city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {"type": "string", 
                             "description": "The name ofthe city to get the weather of",
                    }
                },
                "required": ["city"]
            },
        },
    }
]

In [65]:
# 자동완성을 위해 import
from openai.types.chat import ChatCompletionMessage

def process_ai_response(message: ChatCompletionMessage):
    # AI 모델이 우리한테 툴을 실행하라고 요청하는지 확인(그러한 요청을 메모리에 저장하기 위해)
    if message.tool_calls:
        # tool_call 있는 요청을 메모리에 저장
        # Run_request
        messages.append({
            "role": "assistant",
            "content": message.content or "",
            "tool_calls": [
                {
                    "id": tool_call.id,
                    "type":"function",
                    "function": {
                        "name": tool_call.function.name,
                        "arguments": tool_call.function.arguments,
                    }
                } for tool_call in message.tool_calls
            ]
        })
        
        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments  # 단순 문자열 '{"city": "Spain"}'
            
            # arguments Schema Example: '{"city": "Spain"}'
            print(f"Calling function: {function_name} with {arguments}")

            # JSON string 로드
            # 받은 문자열을 실제 Python 객체(;딕셔너리)로 바꾸는 것
            # ?해당 작업이 디코딩?
            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments = {}
                
            # 호출한 함수를 function_map 내부에서 검색
            function_to_run = FUNCTION_MAP.get(function_name)
            
            result = function_to_run(**arguments)
            
            print(f"Ran {function_name} with args {arguments} for a result of {result}")
            
            # 실행 결과 저장
            # tool_response
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": function_name,
                "content": result,
            })
        
        # AI 다시 호출
        call_ai()
    else:
        messages.append({"role": "assistant", "content": message.content})
        print(f"AI: {message.content}")

def call_ai():
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=TOOLS,
    )
    # Check!
    print(messages)
    process_ai_response(response.choices[0].message)

In [66]:

while True:
    message = input("Send a message to the LLM...")
    if message == "quit" or message == "q":
        # Check!
        print(messages)
        break
    else:
        messages.append({"role": "user", "content": message})
        print(f"User: {message}")
        call_ai()

User: My name is Kim
[{'role': 'user', 'content': 'My name is Kim'}]
AI: Nice to meet you, Kim! How can I assist you today?
User: What is my name?
[{'role': 'user', 'content': 'My name is Kim'}, {'role': 'assistant', 'content': 'Nice to meet you, Kim! How can I assist you today?'}, {'role': 'user', 'content': 'What is my name?'}]
AI: Your name is Kim.
User: What is the weather in Spain?
[{'role': 'user', 'content': 'My name is Kim'}, {'role': 'assistant', 'content': 'Nice to meet you, Kim! How can I assist you today?'}, {'role': 'user', 'content': 'What is my name?'}, {'role': 'assistant', 'content': 'Your name is Kim.'}, {'role': 'user', 'content': 'What is the weather in Spain?'}]
Calling function: get_weather with {"city":"Spain"}
Ran get_weather with args {'city': 'Spain'} for a result of 33 degrees celcius.
[{'role': 'user', 'content': 'My name is Kim'}, {'role': 'assistant', 'content': 'Nice to meet you, Kim! How can I assist you today?'}, {'role': 'user', 'content': 'What is my 

In [69]:
messages

[{'role': 'user', 'content': 'My name is Kim'},
 {'role': 'assistant',
  'content': 'Nice to meet you, Kim! How can I assist you today?'},
 {'role': 'user', 'content': 'What is my name?'},
 {'role': 'assistant', 'content': 'Your name is Kim.'},
 {'role': 'user', 'content': 'What is the weather in Spain?'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'id': 'call_ku0RoQJw64U4qaUckpz6qBTo',
    'type': 'function',
    'function': {'name': 'get_weather', 'arguments': '{"city":"Spain"}'}}]},
 {'role': 'tool',
  'tool_call_id': 'call_ku0RoQJw64U4qaUckpz6qBTo',
  'name': 'get_weather',
  'content': '33 degrees celcius.'},
 {'role': 'assistant',
  'content': 'The weather in Spain is 33 degrees Celsius. If you need more specific information about a particular city in Spain, feel free to ask!'}]